In [1]:
import pandas as pd
import requests
import json
import time

# Read data
data= pd.read_excel('FSOP_Customer_Data_v2.0.xls')

# Get unique latitudes and longitudes
unique_locations = data[['GEO_LATITUDE', 'GEO_LONGITUDE']].drop_duplicates()

# Set up Yelp API authentication
api_keys = ['vSU6Bwx_p3F5lWISBYrupJa-hW7Y-l4nHorywqyTJkuZ1sQIpzfQOxKMpDGTMdvrG3jOKykZ6Ci-OQxNEiNCbAkqpMgoUnZoZVL9nKsyMqxQ_stg9FRuAN62HbQmZHYx','vSU6Bwx_p3F5lWISBYrupJa-hW7Y-l4nHorywqyTJkuZ1sQIpzfQOxKMpDGTMdvrG3jOKykZ6Ci-OQxNEiNCbAkqpMgoUnZoZVL9nKsyMqxQ_stg9FRuAN62HbQmZHYx',
           'L1XtFz_lPa3A57AcWyGI5SXJdvhe1mqKsiw5kW-beiSQDpW_3ZLVIBndyBjipqtf0srx_hbDCd7fm-8pnE2zNTH2t5OlMHCXuAvewTQyd5hrPjWZ2ya4CRug9UslZHYx','TnwHEqRJhcItEl64vABBM9z2HQoYo_fBvCyhRQVtKz2VQiQO5JyRigR9TOgkx5XtUJB7pCtjxk3uCd6MBuKIM_VyxSOaqI-_mdaVv62tBnjeH20nPpUvGPY6wFIlZHYx','YmEvL_xZW_xJPPN4m5y_OKzs2bmWMzQOvwcHorR43jvuVp9Qk6D1ca3QgnFygw_yBTiWo4kX_hFcxr2cl-kTzFVStULA5faejqQogCvUE5x8YGMamo6MWM7nm2IlZHYx','VuKBKG3hweM912gpFFxRWKwyZvOvtnCcLgwgmto46q8EUS5CykxWYA3waONcwsOJ-76lIahc8XzhaO0ptqnFVqLo99oaGZJCeUGj5v_upDhB7oaDWAXnp619Y2wlZHYx',
           'plNDiOGMFNjYnB2fQ9nQLxdAuPLMu-1o5smoXOCfH1tUjH4BvaFSfCAobScD4Mto2TFjTZzM-lf4qt_xA3ikxqZwTnKbaVG31vEwYRE5FdKE8afzG-Jm--Ag2G0lZHYx','yMWSHlNgxUPbDca0Sc0PweHDHtDmoHcL-DQdU4eWNtBtZyqhmTfPuaQpQstp_Its5PE3ZdSg_GVvrZfS_PgwcIJgYnNGH_1AZc2G6H_ztMoyJogRaX_udI4LlPklZHYx','O0D4Mb2Nhoj54nqOUSM4WCXy3NP2c1fD9b-YdI5LZ9HDxLW7kKEv7tY8-MDr71XLqo78Pg3r-8f_zSat3PrOYTZU_9fqCvQD20S6EN1ddYh9f3qUBVOgcLS4HMQmZHYx','46zKSVmx5SGc6oi7P-KX5r1UkP3PF0nWlSIYRA86v-FHsxgiLuE9xIWb8yyPuhuL8ezFX6e1BU2FYkCpYzT9VmWImdkH5dseh6dCKIqQs0TzgP0rYRwF3U87FMUmZHYx']
headers = {
    'Content-Type': 'application/json'
}

# Set up variables for access limit
access_count = 0
access_limit = 5000

# Add ratings column to data
data['rating'] = ''

# Loop through unique locations and get ratings
for index, row in unique_locations.head(50).iterrows():
    latitude = row['GEO_LATITUDE']
    longitude = row['GEO_LONGITUDE']
    
    # Construct API request URL
    api_key = api_keys[access_count % len(api_keys)]
    url = f'https://api.yelp.com/v3/businesses/search?term=food&latitude={latitude}&longitude={longitude}&limit=50'
    
    # Send API request and check status code
    headers['Authorization'] = f'Bearer {api_key}'
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f'Request failed with status code {response.status_code}')
        break
        
    # Parse response as JSON
    response_json = json.loads(response.text)
    
    # Print progress
    print(f'Processing location {index+1} of {unique_locations.shape[0]}: {latitude}, {longitude}')
    print(f'Found {len(response_json["businesses"])} businesses')
    
    # Loop through businesses and get ratings
    for business in response_json['businesses']:
        business_id = business['id']
        rating = 'NA'  # Initialize rating variable
        
        # Check if rating field is present and assign its value to rating variable
        if 'rating' in business:
            rating = business['rating']
            
            # Check if rating is a number between 1.0 and 5.0, otherwise set rating to 'NA'
            if not isinstance(rating, (int, float)) or rating < 1.0 or rating > 5.0:
                rating = 'NA'
        else:
            rating = 'NA'
        
        # Update data frame with rating
        data.loc[(data['GEO_LATITUDE'] == latitude) & (data['GEO_LONGITUDE'] == longitude), 'rating'] = rating
        
        # Print rating for each business
        print(f'Business ID: {business_id}, Rating: {rating}')
    
    # Wait for 1 second to avoid hitting the rate limit
    time.sleep(1)
    
    # Check access count and stop loop if limit is reached
    if access_count >= access_limit:
        print('Access limit reached')
        break

        
# Print data frame with reviews
print(data)

# Save data frame to file
data.to_csv('FSOP_Customer_Data_v2.0_with_rat.csv', index=False)


Processing location 1 of 32061: 32.115937, -110.928378
Found 50 businesses
Business ID: EnKAH7wEUGyLRhSx0x-v0g, Rating: 4.5
Business ID: GlayTPWmrPj8qCaaJuafdg, Rating: 4.0
Business ID: WXX2EtysiTsZd7pzijpb0g, Rating: 3.0
Business ID: 8RAVcQTTGugug3nK39hZ5g, Rating: 3.5
Business ID: G4rgfDp5pltixiUDblEUvg, Rating: 4.5
Business ID: PvClAHXbI_DzGA5mmmUqAg, Rating: 4.5
Business ID: zbhID412Pg3zXd_t3mswpg, Rating: 3.0
Business ID: 2wfNHd2kPK3ioEGhb9QTkw, Rating: 4.5
Business ID: u5FV7F0Hjk6W-NGvxWqPRg, Rating: 4.0
Business ID: 4Z2T5UGalIkjlOIrlBrd8Q, Rating: 4.5
Business ID: UrLO_B_-eIuiJjhlB7UlEg, Rating: 5.0
Business ID: 0Q_A1mtkXkovpuqgyT1BtA, Rating: 5.0
Business ID: -Rmxzme0KxjvRG45p4gqlA, Rating: 3.5
Business ID: lhsQkb5nhf-Kd5OvgB9MNg, Rating: 3.5
Business ID: JFteGsQlrJeJjur6cA1RhA, Rating: 4.5
Business ID: ayWedGvFe1A2AjUpDlnFNA, Rating: 4.5
Business ID: 7tSYalo0aELwl9C_dHgk0A, Rating: 3.5
Business ID: iKLhnstKcsUILIaOHwhJ8Q, Rating: 4.5
Business ID: ZbLroF8vP4IbFR-cd_iI3w, Rating

KeyboardInterrupt: 